### IMPORT DE BIBLIOTECAS

In [1]:
from collections import defaultdict
import pandas as pd
import numpy as np
import os

import implicit
from implicit.evaluation import (train_test_split, 
                                 ndcg_at_k, 
                                 AUC_at_k,
                                 mean_average_precision_at_k, 
                                 precision_at_k)

import recometrics
from cmfrec import MostPopular
from lightfm import LightFM

from scipy.sparse import (csr_matrix, 
                          save_npz, 
                          load_npz)
import scipy

from matplotlib import pyplot as plt
import seaborn as sns

import pickle
from tqdm import tqdm
import warnings

In [2]:
warnings.filterwarnings('ignore')

### Carregamento de dados de categorias e histórico de clicks

In [3]:
df_fr_cat = pd.read_csv(os.getcwd()+'/.csv/new_category_sample.csv') # DataFrame Informativo das categorias

In [4]:
clicks_fr = pd.read_csv(os.getcwd()+'/.txt/clicks_fr_sample.txt', sep = ',', header=0) # df de histórico de clicks

Valores missing na coluna de OfferTitle que serão excluídos.

In [5]:
clicks_fr = clicks_fr[clicks_fr.OfferTitle.isna() == False]

### Conversão das colunas de usuário e oferta em categórica e criando novas colunas com os códigos adotados 

In [6]:
clicks_fr.UserId = clicks_fr.UserId.astype('category')
clicks_fr.OfferId = clicks_fr.OfferId.astype('category')

In [7]:
clicks_fr['User'] = clicks_fr.UserId.cat.codes
clicks_fr['Offer'] = clicks_fr.OfferId.cat.codes

### CRIAÇÃO DE COLUNA DE CLICKS POR CATEGORIA

In [8]:
clicks_fr.drop(columns = ['Keywords'], axis = 1, inplace=True)  # DROP DAS KEYWORDS

#df_fr_cat.rename({'Ancertor_ID':'Ancestor_ID'}, axis = 1, inplace = True) # RENAME DO ANCESTOR_ID

#df_fr_cat.drop('Unnamed: 0', axis =1, inplace = True) # Remoção de coluna Unnamed

clicks_fr['Cat_clicks'] = clicks_fr.groupby('Category')['OfferId'].transform('count') # Criação de coluna de clicks por categoria

In [9]:
df_fr_cat = df_fr_cat[df_fr_cat.Country == 'fr'].drop('Country',axis =1)

### Merge do Dataframe de categorias com o dataframe de clicks 

In [10]:
clicks_fr = clicks_fr.merge(df_fr_cat, on = 'Category')

### Criação de coluna com o nº total de clicks do usuário e nº total de clicks do produto

In [11]:
clicks_fr['UserTotalClicks'] = clicks_fr.groupby(by=['UserId'])['OfferId'].transform('count')

In [12]:
clicks_fr['ProductClicks'] = clicks_fr.groupby(by='OfferId')['UserId'].transform('count')

In [ ]:
def set_params():
    params = []
    for min_product in [20,25, 30, 40, 45, 50]:
        for max_product in [0, 1]:
            for min_users in [5,7, 9, 11]:
                clicks_fr_filtered = clicks_fr[(clicks_fr.ProductClicks > min_product) & (clicks_fr.ProductClicks < (clicks_fr.ProductClicks.mean() + max_product*clicks_fr.ProductClicks.std()))]

                clicks_fr_filtered['UserTotalClicks'] = clicks_fr_filtered.groupby(by=['UserId'])['OfferId'].transform('count')
                clicks_fr_filtered = clicks_fr_filtered[((clicks_fr_filtered.UserTotalClicks > min_users))]

                clicks_fr_filtered.UserId = clicks_fr_filtered.UserId.astype('category')
                clicks_fr_filtered.OfferId = clicks_fr_filtered.OfferId.astype('category')


                clicks_fr_filtered['User'] = clicks_fr_filtered.UserId.cat.codes
                clicks_fr_filtered['Offer'] = clicks_fr_filtered.OfferId.cat.codes

                clicks_per_user_product = clicks_fr_filtered.groupby(by=['User','Offer']).count()['UserTotalClicks'].reset_index().rename({'UserTotalClicks':'UserClicks'}, axis = 1)
                sparse_user_item = csr_matrix((clicks_per_user_product['UserClicks'], 
                                               (clicks_per_user_product['User'], 
                                                clicks_per_user_product['Offer'])))

                sparse_user_item = (sparse_user_item).astype('double')
                kfold = []
                for i in range(3):
                    train, test, model = als_model(sparse_user_item)
                    kfold.append(implicit.evaluation.precision_at_k(model, train, test))
                    precision = np.array(kfold).mean()
                params.append([min_product, max_product, min_users, precision])
                
    full_params = pd.DataFrame(params, columns = ['min_product','max_product', 'min_users', 'precision'])
    return full_params

In [ ]:
#full = set_params()

In [13]:
full.sort_values(by = 'precision', ascending = False)[:10]

NameError: name 'full' is not defined

In [14]:
#clicks_fr_filtered.UserId = clicks_fr_filtered.UserId.astype('category')
# clicks_fr_filtered.OfferId = clicks_fr_filtered.OfferId.astype('category')


# clicks_fr_filtered['User'] = clicks_fr_filtered.UserId.cat.codes
# clicks_fr_filtered['Offer'] = clicks_fr_filtered.OfferId.cat.codes#Cap minimo de clicks para integrar o sistema de recomendação
clicks_fr_filtered = clicks_fr[(clicks_fr.ProductClicks > 40) & (clicks_fr.ProductClicks < (clicks_fr.ProductClicks.mean() + 0.5*clicks_fr.ProductClicks.std()))]

In [15]:
clicks_fr_filtered['UserTotalClicks'] = clicks_fr_filtered.groupby(by=['UserId'])['OfferId'].transform('count')
clicks_fr_filtered = clicks_fr_filtered[(clicks_fr_filtered.UserTotalClicks > 7)]

In [16]:
clicks_fr_filtered.to_csv('fr_model_sample.csv')


### Agrupamento de dos clicks de usuário em ofertas únicas para termos a quantidade de cada usuário em cada oferta.


In [17]:
clicks_per_user_product = clicks_fr_filtered.groupby(by=['User','Offer']).count()['UserTotalClicks'].reset_index().rename({'UserTotalClicks':'UserClicks'}, axis = 1)

In [18]:
len(clicks_per_user_product.User.unique()), len(clicks_per_user_product.Offer.unique())

(788, 1454)

### Criação de matrizes esparsas Usuário-item e item-usuário

In [19]:
alpha = 15
sparse_user_item = csr_matrix((clicks_per_user_product['UserClicks'], (clicks_per_user_product['User'], clicks_per_user_product['Offer'])))

sparse_user_item = (sparse_user_item).astype('double') # Conversão de tipo para que o modelo ALS funcione corretamente
data = (sparse_user_item).astype('double') # Conversão de tipo para que o modelo ALS funcione corretamente

In [20]:
#Esparsidade de matriz
possible_interactions = sparse_user_item.shape[0]*sparse_user_item.shape[1]
interacted = len(sparse_user_item.nonzero()[0])
sparsity = 1 - interacted/possible_interactions
sparsity

0.9999999608725247

 Mais de 99.99% das interações possíveis entre usuários e produtos na atual base dados não foi ainda realizada. Segundo artigo: For collaborative filtering to work, the maximum sparsity you could get away with would probably be about 99.5% or so. Devemos reavaliar a matriz?

In [21]:
save_npz(os.getcwd()+"/.npz/fr/sparse_user_item.npz", sparse_user_item)

In [22]:
model_path = os.getcwd()+'/.pkl/fr/fr_als_model.pkl'

* Criação de diferentes matrizes esparsas para operar com o algoritmo. Usuário-item e item-usuário. Cada uma deve ser usada no momento preciso
* O alfa é o coeficiente de confiabilidade da interação do usuário com um item específico. Valor utilizado fi adotado com base no artigo: https://towardsdatascience.com/alternating-least-square-for-implicit-dataset-with-code-8e7999277f4b. Mas, podemos testar outros valores na validação do modelo.
* Outro artigo de base pra elaboração do modelo: https://medium.com/analytics-vidhya/implementation-of-a-movies-recommender-from-implicit-feedback-6a810de173ac

# FUNÇÃO DE RECOMENDAÇÕES - IMPLICIT

## Treinamento de modelos

In [23]:
offers = pickle.load(open(os.getcwd()+"/.pkl/fr/offers.pkl", "rb"))

Carregamentodo dicionário que converte os códigos de ofertas para o seu título de oferta. Ainda falta traduzir do alemão para o inglês para tirar mais significado dos resultados

In [24]:
def als_model(sparse_user_item):
    
    '''computes p@k and map@k evaluation metrics and saves model'''
    
    #sparse_item_user = load_npz(os.getcwd()+"/.npz/sparse_item_user.npz")
      
    train, test = implicit.evaluation.train_test_split(sparse_user_item, train_percentage=0.8)

    model = implicit.als.AlternatingLeastSquares(factors=10, 
                                                 regularization=0.1, 
                                                 iterations=75,
                                                 calculate_training_loss=False)
    alpha=60
    model.fit(train*alpha)

    with open(model_path, 'wb') as pickle_out:
        pickle.dump(model, pickle_out)
    
    return train, test, model

In [25]:
train, test, model = als_model(sparse_user_item)
implicit.evaluation.precision_at_k(model,train,test)

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/653 [00:00<?, ?it/s]

0.15876923076923077

In [138]:
baseline_precision_at_k(sparse_user_item.T.tocsr())

0.07713675213675197

### Avaliação do modelo

In [26]:
def model_evaluation(train, test, model, k=10): 
    
    '''Avaliação do modelo treinado com as funções da biblioteca Implicit.
    Retorna dicionário com p@k, map@k, ndcg@k e auc@k.'''

    
    p_at_k = implicit.evaluation.precision_at_k(model, train_user_items=train, 
                                                test_user_items=test,
                                                K=k, 
                                                show_progress = False)
    
    m_at_k = implicit.evaluation.mean_average_precision_at_k(model, 
                                                             train_user_items = train, 
                                                             test_user_items = test, 
                                                             K=k, 
                                                             show_progress = False)

    ndcg_at_k = implicit.evaluation.ndcg_at_k(model, 
                                              train_user_items = train,
                                              test_user_items = test, 
                                              K=k, 
                                              show_progress = False)

    auc_at_k = implicit.evaluation.AUC_at_k(model, 
                                            train_user_items = train, 
                                            test_user_items = test, 
                                            K=k, 
                                            show_progress = False)
    metrics = {'p@K':p_at_k, 
               'map@K': m_at_k, 
               'ndcg@K':ndcg_at_k, 
               'auc@K':auc_at_k}
    
    return metrics

In [27]:
metrics = model_evaluation(train, test, model, k=10)
metrics

{'p@K': 0.15876923076923077,
 'map@K': 0.052084692691097216,
 'ndcg@K': 0.0944138268820112,
 'auc@K': 0.5809906988178057}

## Avaliação Recometrics

In [28]:
X_train, X_test, users_test = \
    recometrics.split_reco_train_test(
        sparse_user_item, split_type="joined",
        users_test_fraction = None,
        max_test_users = 10000,
        items_test_fraction = 0.3
    )

In [29]:
### Random recommendations (random latent factors)
rng = np.random.default_rng(seed=1)
UserFactors_random = rng.standard_normal(size=(X_test.shape[0], 5))
ItemFactors_random = rng.standard_normal(size=(X_test.shape[1], 5))

### Non-personalized recommendations
model_baseline = MostPopular(implicit=True, user_bias=False).fit(X_train.tocoo())
item_biases = model_baseline.item_bias_
item_biases

array([0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
       0.0000000e+00, 3.5333572e-05])

In [35]:
### Fitting WRMF model
wrmf = implicit.als.AlternatingLeastSquares(factors=10, regularization=0.1, iterations = 75, random_state=123)
wrmf.fit(X_train*60)

### Fitting BPR model with WARP loss
bpr_warp = LightFM(no_components=50, loss="warp", random_state=123)
bpr_warp.fit((X_train*60).tocoo())

  0%|          | 0/75 [00:00<?, ?it/s]

In [60]:
k = 10 ## Top-K recommendations to evaluate

metrics_random = recometrics.calc_reco_metrics(
    X_train[:X_test.shape[0]], X_test,
    UserFactors_random, ItemFactors_random,
    k=k, all_metrics=True
)

metrics_baseline = recometrics.calc_reco_metrics(
    X_train[:X_test.shape[0]], X_test,
    None, None, item_biases=item_biases,
    k=k, all_metrics=True
)

metrics_wrmf = recometrics.calc_reco_metrics(
    X_train[:X_test.shape[0]], X_test,
    wrmf.user_factors[:X_test.shape[0]], wrmf.item_factors,
    k=k, all_metrics=True
)

metrics_bpr_warp = recometrics.calc_reco_metrics(
    X_train[:X_test.shape[0]], X_test,
    bpr_warp.user_embeddings[:X_test.shape[0]], bpr_warp.item_embeddings,
    item_biases=bpr_warp.item_biases,
    k=k, all_metrics=True
)

In [61]:
all_metrics = [
    metrics_random,
    metrics_baseline,
    metrics_wrmf,
    metrics_bpr_warp
]
all_metrics = pd.concat([m.mean(axis=0).to_frame().T for m in all_metrics], axis=0)
all_metrics.index = [
    "Random",
    "Non-personalized",
    "WRMF (a.k.a. iALS)",
    "BPR-WARP"
]
all_metrics

,P@10,TP@10,R@10,AP@10,TAP@10,NDCG@10,Hit@10,RR@10,ROC_AUC,PR_AUC
Random,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.504914,0.000045
Non-personalized,0.008924,0.027618,0.027560,0.007317,0.007331,0.016578,0.077428,0.021008,0.962788,0.017306
WRMF (a.k.a. iALS),0.047900,0.159246,0.158779,0.061654,0.061825,0.107008,0.388451,0.146096,0.904198,0.095503
BPR-WARP,0.012336,0.037503,0.037274,0.011194,0.011304,0.024004,0.106299,0.034085,0.963917,0.022720


In [62]:
all_metrics = all_metrics.apply(lambda x: round(x,3))
all_metrics

,P@10,TP@10,R@10,AP@10,TAP@10,NDCG@10,Hit@10,RR@10,ROC_AUC,PR_AUC
Random,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.505,0.000
Non-personalized,0.009,0.028,0.028,0.007,0.007,0.017,0.077,0.021,0.963,0.017
WRMF (a.k.a. iALS),0.048,0.159,0.159,0.062,0.062,0.107,0.388,0.146,0.904,0.096
BPR-WARP,0.012,0.038,0.037,0.011,0.011,0.024,0.106,0.034,0.964,0.023


In [63]:
all_metrics.to_csv('.csv/fr/fr_all_metrics_10.csv')

Fonte: https://nb.recohut.com/implicit/music/evaluation/2021/07/13/evaluation-recometrics-lightfm.html

* Sobre metricas de precisão @k: https://medium.com/@m_n_malaeb/recall-and-precision-at-k-for-recommender-systems-618483226c54
* Sobre NDCG: https://towardsdatascience.com/evaluate-your-recommendation-engine-using-ndcg-759a851452d1
* Sobre Mean Average Precision: https://towardsdatascience.com/breaking-down-mean-average-precision-map-ae462f623a52

Ainda não foi realizada qualquer tunagem de hiperaparâmetros. Podemos pegar alguns valores de referencia para rodar um gridsearch

## Funções de recomendações

In [143]:
def recommend(user, K=10):
    
    ''' Retorna uma lista de itens recomendados para o usuário dado de acordo com a biblioteca Implicit.
        Também é retornado uma lista com os itens já clicados por esse usuário'''
    
    #sparse_user_item = load_npz("/.npz/sparse_user_item.npz")
    
    with open(model_path, 'rb') as pickle_in:
        model = pickle.load(pickle_in)
    
    if user not in sparse_user_item.T.tocsr().indices:
        return "Invalid User"
        
    recommended, scores = (model.recommend(user, sparse_user_item[user], K))

    original_user_items = list(sparse_user_item[user].indices)

    return recommended, original_user_items

* A matriz m passa a ser a matriz com o novo usuário atualizado e é levada em consideração no para o cálculo de novos vetores.

Nota: 
* Após os ajustes na organização das matrizes esparsas, o modelo parece não mais repetir recomendações de itens que já foram clicados pelo usuário
* O modelo parece também não mais necessitar de tradução dos códigos de ofertas e usuário adotados na matriz esparsa para os códigos da matriz original

### Criação e armazenamento do dicionário código-titulo de oferta (carregado na parte de cima do código).

In [186]:
df_temp = clicks_fr[['Offer','OfferTitle']]

In [187]:
df_temp = df_temp.drop_duplicates(['Offer','OfferTitle'])

In [191]:
offers = dict(zip(df_temp['Offer'], df_temp['OfferTitle']))

In [192]:
#dic = pd.read_csv('de_traduzido.csv')
#dic['Offer2'] = dic['OfferTitle'].map(dict(zip(clicks_fr.OfferTitle, clicks_fr.Offer)))

In [193]:
pickle.dump(offers, open('.pkl/fr/offers.pkl', 'wb'))

### Criação e armazenamento do dicionário código_oferta -> Filha_1 e código_oferta -> Categoria (carregado na parte de cima do código).

In [194]:
clicks_fr.filha_1 = clicks_fr.filha_1.astype('int32')

In [195]:
df_temp = clicks_fr[['Offer','filha_1', 'Category']]

In [196]:
df_temp = df_temp.drop_duplicates(['Offer','filha_1', 'Category'])

In [201]:
offer_filha1 = dict(zip(df_temp['Offer'], df_temp['filha_1']))
pickle.dump(offer_filha1, open('.pkl/fr/offer_sub1.pkl', 'wb'))

In [202]:
offer_cat = dict(zip(df_temp['Offer'], df_temp['Category']))
pickle.dump(offer_cat, open('.pkl/fr/offer_cat.pkl', 'wb'))

### Tabela de info dos produtos

In [203]:
products_info = clicks_fr[['Offer','Category','filha_1', 'ProductClicks']]

In [204]:
products_info.filha_1 = products_info.filha_1.astype('int32')

In [205]:
products_info = products_info.drop_duplicates().reset_index().drop('index', axis = 1)

In [206]:
products_info.rename({'filha_1':'SuperCat'}, axis = 1, inplace = True)

In [207]:
products_info.sort_values(by=['SuperCat','ProductClicks'], ascending = False, inplace = True)

In [222]:
products_info.to_csv('.csv/fr/products_info.csv')

In [209]:
products_info.head()

,Offer,Category,SuperCat,ProductClicks
119765,197556,100513123,100450123,406
305519,44911,100508723,100450123,391
127018,45255,100334923,100450123,373
126916,70579,100334923,100450123,333
238397,225605,100450223,100450123,284


### Dicionário Nome Sub1

In [211]:
df = clicks_fr[['filha_1','filha_1_name']].drop_duplicates()
sub1_name = dict(zip(df.filha_1, df.filha_1_name))
pickle.dump(sub1_name, open('.pkl/fr/sub1_name.pkl', 'wb'))

### Dicionário Nome Cat

In [213]:
df = clicks_fr[['Category','Translate']].drop_duplicates()
cat_name = dict(zip(df.Category, df.Translate))
pickle.dump(cat_name, open('.pkl/fr/cat_name.pkl', 'wb'))


## Tunagem de hiperparametros

In [223]:
grid = {'factors': [10,50,75,100],
       'regularization':[0.1, 0.01],
        'iterations':[50, 75, 100], 
        'alphas':[40,60,80]}

In [224]:
model = implicit.als.AlternatingLeastSquares()

In [225]:
def gridsearch_als(grid):
    results = []
    for factor in grid['factors']:
        for regularization in grid['regularization']:
            for iteration in grid['iterations']:
                for alpha in grid['alphas']:
            
                    model = implicit.als.AlternatingLeastSquares(factors = factor, 
                                                                 regularization = regularization, 
                                                                 iterations = iteration)
                    model.fit(train*alpha)

                    partial = model_evaluation(train, test, model, )
                    results.append([factor, regularization, iteration, alpha, partial['p@K'],partial['map@K'], partial['ndcg@K'], partial['auc@K']])
                
    final = pd.DataFrame(results, columns = ['Factors','Regularization','Iteration','alpha','P@K','MAP@K','NDCG@K','AUC@K'])
    return final
                

In [ ]:
grid_results = gridsearch_als(grid)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

In [248]:
grid_results.to_csv('params.csv')

In [221]:
grid_results.sort_values(by=['P@K','NDCG@K'], ascending = False)

,Factors,Regularization,Iteration,alpha,P@K,MAP@K,NDCG@K,AUC@K
47,10,0.10,75,60,0.165091,0.052673,0.096149,0.582294
63,10,0.01,75,60,0.161941,0.051783,0.094654,0.579990
50,10,0.01,10,40,0.161311,0.058867,0.101609,0.582347
14,10,1.00,75,40,0.160681,0.053831,0.096934,0.581091
1,10,1.00,10,15,0.160681,0.051685,0.094968,0.578882
...,...,...,...,...,...,...,...,...
235,100,0.10,50,60,0.129805,0.057763,0.091432,0.566779
206,100,1.00,75,40,0.128544,0.056881,0.091341,0.567603
255,100,0.01,75,60,0.127914,0.058619,0.091542,0.566146
207,100,1.00,75,60,0.126024,0.059046,0.091733,0.565739
